In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
# all these urls are breweries' beer lists
urls = ['https://untappd.com/VanishBeer/beer','https://untappd.com/TrilliumBrewing/beer']#, 
#         'https://untappd.com/treehousebrewco/beer',
#         'https://untappd.com/SaltyTurtleBeerCompany/beer', 'https://untappd.com/BearChaseBrewingCompany/beer','https://untappd.com/TexasLeaguer/beer',
#         'https://untappd.com/WillowParkBrewing/beer','https://untappd.com/PipsMeadery/beer','https://untappd.com/EskerHartBrewingCompany/beer',
#         'https://untappd.com/prolyfk/beer','https://untappd.com/Marlobobo/beer', 'https://untappd.com/EmperorsBrewery/beer',
#         'https://untappd.com/SideProject/beer', 'https://untappd.com/troobado/beer', 'https://untappd.com/w/private-press-brewing/472291/beer', 
#         'https://untappd.com/w/fidens-brewing-co/426930/beer', 'https://untappd.com/MindfulAles/beer', 'https://untappd.com/rootandbranchbrewing/beer'
#         'https://untappd.com/WeaverHollowBrewery/beer', 'https://untappd.com/FreakFolkBier/beer', 'https://untappd.com/HorusAles/beer', 
#         'https://untappd.com/CaseyBrewingBlending/beer', 'https://untappd.com/HouseofFermentology/beer', 'https://untappd.com/treehousebrewco/beer', 
#         'https://untappd.com/barclaybrewingco/beer', 'https://untappd.com/Alchemist_Beer/beer', 'https://untappd.com/HillFarmsteadBrewery/beer',
#         'https://untappd.com/monkishbrewing/beer','https://untappd.com/topplinggoliathbrewing/beer', 'https://untappd.com/OldNationBrewingCompany/beer',
#         'https://untappd.com/PrairieArtisanAles/beer', 'https://untappd.com/fiddlehead/beer', 'https://untappd.com/NewEnglandBrewingCo/beer', 
#         'https://untappd.com/fremontbrewing/beer', 'https://untappd.com/RevisionBrewingCompany/beer', 'https://untappd.com/JackieOsBrewery/beer',
#         'https://untappd.com/westbrookbrewing/beer', 'https://untappd.com/creaturecomforts/beer', 'https://untappd.com/LaCumbreBrewingCompany/beer',
#         'https://untappd.com/SurlyBrewingCompany/beer','https://untappd.com/pfriembeer/beer','https://untappd.com/RevolutionBrewingChicago/beer',
#         'https://untappd.com/NewGlarusBrewingCompany/beer','https://untappd.com/halfacrebeer/beer','https://untappd.com/TheStarkeller/beer',
#         'https://untappd.com/HenHouseBrewing/beer','https://untappd.com/MelvinBrewing/beer','https://untappd.com/AlpineBeerCo/beer',
#         'https://untappd.com/Rhinegeist/beer','https://untappd.com/hardywood/beer', 'https://untappd.com/portcitybrewing/beer'] 

In [3]:
print('Number of different Breweries: ',len(urls))
#I accidenty added a meadery, that can either be good or bad tbh but just noting tgis so I don't forget it

Number of different Breweries:  2


In [8]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

### Show details
scrolls through the page and clicks 'show more' to get the full description

In [2]:
def show_details(items):
    #this loop goes through all the beers in the list and clicks "show more" to get the full description
    for i in range(len(items)):
        beer_det = items[i].find_element_by_css_selector('div.beer-details')
        beer_det.location_once_scrolled_into_view
        try:
            beer_det.find_element_by_css_selector('a.read-more-beerlist.track-click').click()
        except NoSuchElementException:
            continue
        beer_det.location_once_scrolled_into_view

### Get Beer Details
This function goes through the list of beers and gets all the information from them and puts them in a list

In [41]:
def get_beer_details(items):
    for beer in items:   #goes through each beer in the "list" and gets the name, style, IBU, abv, and description (half)
        
        #goes through the beer details and grabs the attributes
        try: 
            name = beer.find_element_by_css_selector('p.name').text    #extracts name of beer
            ###style = beer.find_element_by_css_selector('p.style').text   #extracts style of beer
            style = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[3]').text   #replaced line above when using the new method
            desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[4]').text[:-11]   #gets the full description and also cuts off the 'show more'
            
            if (desc=='')|(desc==' '): #if desc isn't long enough to have a "read More" then get the other desc which is stored where the half one is
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[3]').text
                if (desc=='')|(desc==' '):    # in some edge cases the value desc is still going to be blank
                    print('test')
                    desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[5]').text
            
            if 'no longer being produced' in desc:
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[6]').text   # if the beer is no longer being produced
            abv = beer.find_element_by_css_selector('div.details-item.abv').text   #extracts abv of beer
            ibu = beer.find_element_by_css_selector('div.details-item.ibu').text   #extracts ibu of beer
            
        # sometimes the namesoace is different for abv and ibu this will catch that
        except NoSuchElementException:
            abv = beer.find_element_by_css_selector('p.abv').text   # part of the way through the process the namespace 
            ibu = beer.find_element_by_css_selector('p.ibu').text   # changes for ibu and abv so this handles that

        name_ls.append(name)     #these add all the attributes to a list that will eventually be set to a df
        style_ls.append(style)
        desc_ls.append(desc)    
        abv_ls.append(abv)             
        ibu_ls.append(ibu)

        print("Name: ", name,'\nStyle:',style,'\nABV: ',abv,'\nIBU: ',ibu,'\nDescription: ',desc,'\n\n------------\n')

#### Brewery scrape
This cell goes through the list of urls (breweries) and gets their beers

In [23]:
for i in range(len(urls)):   #goes through all the urls/breweries
    url=urls[i]    #sets url to new brewery
    driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
    driver.get(url)   #activates url (brings website up)
    beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
    
    show_details(beers)  #method clicks show more on each beer
    get_beer_details(beers)

C:\Users\612456\AppData\Local\Temp\2\ipykernel_1192\1246772369.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
C:\Users\612456\AppData\Local\Temp\2\ipykernel_1192\1246772369.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers


Name:  Ghost Fleet 
Style: IPA - American 
ABV:  7.1% ABV 
IBU:  N/A IBU 
Description:  This is our house IPA. It is hugely hoppy with fruity aromatics from Amarillo and Simcoe hops. A hazy and unfiltered beer with a smooth finish. 

------------

Name:  Fat Boys 
Style: IPA - Imperial / Double 
ABV:  8.5% ABV 
IBU:  N/A IBU 
Description:  This is our Double IPA. It features bold fruity aromatics commonly associate with American IPAs. It contains a ton of hops including Citra and Equinox. As far as DIPAs go this one is light-bodied with an easy finish 

------------

Name:  Wraith 
Style: Wheat Beer - Witbier / Blanche 
ABV:  5.3% ABV 
IBU:  N/A IBU 
Description:  Wraith is a Belgian Style Witbier that provides you with notes of citrus, chamomile and coriander. 

------------

Name:  Lucketts Calling 
Style: IPA - American 
ABV:  6.3% ABV 
IBU:  N/A IBU 
Description:  A hazy IPA with big fruity hop flavors and almost no bitterness. 

------------

Name:  Beyond The Farm 
Style: IPA - A

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: unable to send message to renderer
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0029B8F3+2406643]
	Ordinal0 [0x0022AF31+1945393]
	Ordinal0 [0x0011C748+837448]
	Ordinal0 [0x0010C9B3+772531]
	Ordinal0 [0x0010CDC8+773576]
	Ordinal0 [0x00118BE0+822240]
	Ordinal0 [0x0010E0BD+778429]
	Ordinal0 [0x0010EC9A+781466]
	Ordinal0 [0x0010E2CA+778954]
	Ordinal0 [0x0010D8A6+776358]
	Ordinal0 [0x0010C7C5+772037]
	Ordinal0 [0x0010CC9D+773277]
	Ordinal0 [0x0011DF4A+843594]
	Ordinal0 [0x00174D3D+1199421]
	Ordinal0 [0x0016427C+1131132]
	Ordinal0 [0x00174682+1197698]
	Ordinal0 [0x00164096+1130646]
	Ordinal0 [0x0013E636+976438]
	Ordinal0 [0x0013F546+980294]
	GetHandleVerifier [0x00509612+2498066]
	GetHandleVerifier [0x004FC920+2445600]
	GetHandleVerifier [0x00334F2A+579370]
	GetHandleVerifier [0x00333D36+574774]
	Ordinal0 [0x00231C0B+1973259]
	Ordinal0 [0x00236688+1992328]
	Ordinal0 [0

#### the above cell takes ~40 mins to run 50 breweries

### Data is recorded in the lists now add to df

In [35]:
#create df
df = pd.DataFrame()
df['Name'] = name_ls
df['style'] = style_ls
df['IBU'] = ibu_ls
df['ABV'] = abv_ls
df['Description'] = desc_ls
df

,Name,style,IBU,ABV,Description
0,Längst (Blend #1),Adambier,N/A IBU,10% ABV,Längst is our Bourbon barrel-aged Adambier - a...
1,Adam from the Wood,Adambier,N/A IBU,12% ABV,Adam From The Wood is our Adam aged in America...
2,Cherry Adam from the Wood,Adambier,N/A IBU,13.5% ABV,Cherry Adam from the Wood is aged with locally...
3,Adam from the Wood - Rye (2015),Adambier,N/A IBU,12.4% ABV,
4,Bishop's Barrel No. 17,Adambier,30 IBU,13.2% ABV,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
...,...,...,...,...,...
2785,Grätzer (Signature Series),Grodziskie / Grätzer,N/A IBU,4% ABV,Grodziski
2786,Polish,Grodziskie / Grätzer,N/A IBU,3.4% ABV,Grodziski
2787,Grätzer,Grodziskie / Grätzer,30 IBU,4.1% ABV,Grodziski
2788,Piwo Grodziskie,Grodziskie / Grätzer,27 IBU,3.5% ABV,Grodziski


In [36]:
#get rid of all beers that don't have all 3 major things I need
df = df[(df['IBU']!='N/A IBU')&(df['ABV']!='N/A ABV')&(df['Description']!='')]

In [37]:
df

,Name,style,IBU,ABV,Description
4,Bishop's Barrel No. 17,Adambier,30 IBU,13.2% ABV,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
5,Adam,Adambier,50 IBU,10% ABV,Adam is a recreation of a historic beerstyle. ...
8,Dortmunder Adambier,Adambier,50 IBU,12.7% ABV,"A recreation of a historical German style, the..."
10,Arcanum (Singularis #1) (Hillrock Estate Soler...,Adambier,46 IBU,11% ABV,Arcanum is our Adambier. Adambier is an extinc...
11,Thurston Adambier,Adambier,18 IBU,12% ABV,Germany has always brewed; and before their la...
...,...,...,...,...,...
2783,Grodziskie,Grodziskie / Grätzer,35 IBU,3.8% ABV,Grodziski
2784,Grodziskie,Grodziskie / Grätzer,30 IBU,3.8% ABV,Grodziski
2787,Grätzer,Grodziskie / Grätzer,30 IBU,4.1% ABV,Grodziski
2788,Piwo Grodziskie,Grodziskie / Grätzer,27 IBU,3.5% ABV,Grodziski


In [38]:
df['IBU'] = df['IBU'].str[:-4]
df['ABV'] = df['ABV'].str[:-5]
df['ABV'] = df['ABV'].astype(float)
df['IBU'] = df['IBU'].astype(int)

C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1187835521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].str[:-4]
C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1187835521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].str[:-5]
C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1187835521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [39]:
df

,Name,style,IBU,ABV,Description
4,Bishop's Barrel No. 17,Adambier,30,13.2,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
5,Adam,Adambier,50,10.0,Adam is a recreation of a historic beerstyle. ...
8,Dortmunder Adambier,Adambier,50,12.7,"A recreation of a historical German style, the..."
10,Arcanum (Singularis #1) (Hillrock Estate Soler...,Adambier,46,11.0,Arcanum is our Adambier. Adambier is an extinc...
11,Thurston Adambier,Adambier,18,12.0,Germany has always brewed; and before their la...
...,...,...,...,...,...
2783,Grodziskie,Grodziskie / Grätzer,35,3.8,Grodziski
2784,Grodziskie,Grodziskie / Grätzer,30,3.8,Grodziski
2787,Grätzer,Grodziskie / Grätzer,30,4.1,Grodziski
2788,Piwo Grodziskie,Grodziskie / Grätzer,27,3.5,Grodziski


OLD CSV

In [ ]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('untappd_beers.csv')

In [240]:
# look at how I can get more beers for 
###Wheat Beer 207 more
###Porter 207 more
###Belgian 203 more
###Lager 191 more
###Pale Ale 178 more
###Wild/Sour Ale 155 more
###Stout 126 more

### new method to get more beers that aren't IPAs

In [67]:
url = 'https://untappd.com/beer/top_rated'
driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
driver.get(url)   #activates url (brings website up)
#chrome_options.add_argument('--disable-dev-shm-usage')

C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1997648390.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver


In [43]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

In [ ]:
# don't reset the vaues of the lists

In [ ]:
# after the first run it went from 1 to 70(historical beer)
# next run we are going from 71  to as far as we can (101)
# next run we are going from 102  to as far as we can ()

In [68]:
#selects just the american beers
country = driver.find_element_by_css_selector('select#sort_picker')
country.click()
style = country.find_element_by_xpath('//*[@id="sort_picker"]/option[208]')
style.click()

# #this goes through each of the styles, clicks on them and skips IPAs
for i in range(102,226):
    item = 'option['+str(i)+']'
    menu = driver.find_element_by_css_selector('select#filter_picker')
    driver.execute_script("arguments[0].click();", menu)
    #menu.click()  #clicks on the drop down menu
    style = menu.find_element_by_xpath('//*[@id="filter_picker"]/'+item)
    if 'IPA' not in style.text:
        style.click()
        beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
        show_details(beers)
        get_beer_details(beers)
    element = WebDriverWait(driver, 10).until(   #waits to continue until the drop down menu is located
        EC.presence_of_element_located((By.ID, "filter_picker")))
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  #scrolls back to the top of the page to click the next style

C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\3714239034.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  country = driver.find_element_by_css_selector('select#sort_picker')
C:\Users\612456\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  warnings.warn("find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead")
C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\3714239034.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  menu = driver.find_element_by_css_selector('select#filter_picker')
C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\3714239034.py:16: DeprecationWarning: find_elements_by

Name:  Dwell (Batch 1) 
Style: Lager - American Light 
ABV:  5% ABV 
IBU:  N/A IBU 
Description:  Dwell is our koelschip open fermented lager beer! A unique and special beer for us, Dwell was the last beer brewed on our original brewhouse
We pumped lager wort into our koelschip on a cold winter day, pitched yeast and allowed the beer to open ferment for 8 days. We then transferred the beer into oak barrels and pushed them into our walk-in cooler for over three months of lager conditioning.
We then bottled Dwell at our Fünkhaust and allowed it to bottle condition for over seven months. Crisp like a lager with hints of oak and wild funk! Read Less 

------------

Name:  Domestic 
Style: Lager - American Light 
ABV:  4.2% ABV 
IBU:  N/A IBU 
Description:  Domestic Light Lager Honeysuckle • Wafer Cookie • Crisp Finish ABV: 4.2 

------------

Name:  Coffee Lager 
Style: Lager - American Light 
ABV:  4.5% ABV 
IBU:  15 IBU 
Description:  Lager - American Light 

------------

Name:  SOAKER 

C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\3714239034.py:21: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  #scrolls back to the top of the page to click the next style


Name:  Rhetorica 
Style: Lager - Dark 
ABV:  5% ABV 
IBU:  N/A IBU 
Description:  Lager - Dark 

------------

Name:  River 
Style: Lager - Dark 
ABV:  4.4% ABV 
IBU:  N/A IBU 
Description:  Czech-style B 

------------

Name:  Massacre Bourbon Barrel Aged (2015) 
Style: Lager - Dark 
ABV:  13% ABV 
IBU:  80 IBU 
Description:  Bourbon-barrel-aged Imperial Dark Lager. We brew this every year and age it in freshly dumped bourbon barrels for months to achieve the smooth, balanced, silky, bourbon-infused malty richness that is Massacre. 2015 was our fourth year brewing, bottling, and wax-dipping each and every bottle. Complex, featuring dried stone fruits, chocolate, and a sturdy alcohol backbone. Read Less 

------------

Name:  Quiet Night 
Style: Lager - Dark 
ABV:  5.3% ABV 
IBU:  N/A IBU 
Description:  Lager - Dark 

------------

Name:  The Smoothness 
Style: Lager - Dark 
ABV:  8.7% ABV 
IBU:  N/A IBU 
Description:  Lager - Dark 

------------

Name:  Bourbon Barrel Aged Tripper Boc

Name:  Foeder Aged Coal Hill 
Style: Lager - Dortmunder / Export 
ABV:  5.4% ABV 
IBU:  N/A IBU 
Description:  Dortmunder lager aged in an oak foeder for two months. Delicate floral and citrus undertones backed up by toasted marshmallow, caram 

------------

Name:  Gold Noise 
Style: Lager - Dortmunder / Export 
ABV:  5.5% ABV 
IBU:  N/A IBU 
Description:  Lager - Dortmunder / Export 

------------

Name:  Export Lager 
Style: Lager - Dortmunder / Export 
ABV:  5.7% ABV 
IBU:  20 IBU 
Description:  Lager - Dortmunder / Export 

------------

Name:  Union Lager 
Style: Lager - Dortmunder / Export 
ABV:  5.2% ABV 
IBU:  19 IBU 
Description:  Lager - Dortmunder / Export 

------------

Name:  Dortmunder 
Style: Lager - Dortmunder / Export 
ABV:  5.5% ABV 
IBU:  25 IBU 
Description:  Award winning, easy drinking German style Dortmunder lager. Best enjoyed anytime, preferably amo 

------------

Name:  Export Lager 
Style: Lager - Dortmunder / Export 
ABV:  5.7% ABV 
IBU:  20 IBU 
Descript

...

Name:  Orviamo 
Style: Wild Ale - American 
ABV:  10% ABV 
IBU:  N/A IBU 
Description:  Collaboration with de Garde. Wild ale with microflora from both breweries, Oregon raspberries and plums, then aged in a combination of wine, whiskey and port barrels. Read Less 

------------

Name:  Atrial Rubicite 
Style: Wild Ale - American 
ABV:  5.8% ABV 
IBU:  N/A IBU 
Description:  Wild Ale - American 

------------

Name:  Veritas 015 
Style: Wild Ale - American 
ABV:  6.5% ABV 
IBU:  N/A IBU 
Description:   

------------

Name:  Elements of Composition (Batch 1) 
Style: Wild Ale - American 
ABV:  6.2% ABV 
IBU:  N/A IBU 
Description:  We’re pleased to introduce Elements of Composition, a blend of beer and friendship spanning the course of years. In February 2016, brewers from De Garde Brewing, Jester King Brewery, and Sante Adairius Rustic Ales in met in Tillamook, Oregon to blend beer from each of their respective breweries. Three year old, 100% spontaneously fermented beer from De Garde 

Name:  Nelson Hose 
Style: Wild Ale - Other 
ABV:  5% ABV 
IBU:  N/A IBU 
Description:  A dry hopped spontaneous wild ale aged in oak barrels with coriander, orange peel an 

------------

Name:  Along the Way 
Style: Wild Ale - Other 
ABV:  9% ABV 
IBU:  N/A IBU 
Description:  Wild Ale - Other 

------------

Name:  SHADOW FACTORY: Cranberry - Boysenberry 
Style: Wild Ale - Other 
ABV:  5.5% ABV 
IBU:  9 IBU 
Description:  Wild Ale - Other 

------------

Name:  Fated Farmer Black Currant (2019) 
Style: Wild Ale - Other 
ABV:  6.5% ABV 
IBU:  N/A IBU 
Description:  Fated Farmer: Black Currant pours a deep plum color and wastes no time in presenting it's complex, vinous bouquet of aromas. Notes of bold, structured red wine, blackberry preserves, and red grape are the highlights, with underlying earthiness in the background. Dry and slightly tannic, Fated Farmer: Black Currant finishes with vibrant, mouthwatering acidity. Read Less 

------------

Name:  Perpetual Dot 
Style: Wild Ale -

Name:  Drupes 
Style: Wild Ale - Other 
ABV:  8% ABV 
IBU:  N/A IBU 
Description:  Wild Ale - Other 

------------

Name:  Untitled #5 
Style: Wild Ale - Other 
ABV:  6.9% ABV 
IBU:  N/A IBU 
Description:  Wild Ale - Other 

------------

Name:  Bianca Raspberry & Vanilla Sherbet Wild Wild Brett 
Style: Wild Ale - Other 
ABV:  4.6% ABV 
IBU:  N/A IBU 
Description:  Wild Ale fermented in oak foeders with raspberry, vanilla 

------------

Name:  (512) Wild Bear 
Style: Wild Ale - Other 
ABV:  8.5% ABV 
IBU:  9 IBU 
Description:  Wild Ale - Other 

------------

Name:  Coffee Cinnamon Bourbon Abominable (2015) 
Style: Winter Ale 
ABV:  14% ABV 
IBU:  N/A IBU 
Description:   

------------

Name:  Coconut Edition B-Bomb (2019) 
Style: Winter Ale 
ABV:  13.2% ABV 
IBU:  65 IBU 
Description:   

------------

Name:  Ancient One B-bomb (2016) 
Style: Winter Ale 
ABV:  14% ABV 
IBU:  N/A IBU 
Description:  A blend of B-Bomb aged in 35 year Heaven Hills barrels for 30 months, and 15 year barre

Name:  Krooked Kringle 
Style: Winter Ale 
ABV:  7% ABV 
IBU:  20 IBU 
Description:  Winter Ale 

------------

Name:  Barrel Aged Christmas Ale (2020) 
Style: Winter Ale 
ABV:  7.5% ABV 
IBU:  30 IBU 
Description:   

------------

Name:  Kentucky Christmas 
Style: Winter Ale 
ABV:  8% ABV 
IBU:  70 IBU 
Description:  Bourbon barrel aged version of Abominable 

------------

Name:  Holiday Ale 
Style: Winter Ale 
ABV:  9% ABV 
IBU:  N/A IBU 
Description:  Winter Ale 

------------

Name:  Rye Whiskey Very Ill-Tempered Gnome 
Style: Winter Ale 
ABV:  12% ABV 
IBU:  65 IBU 
Description:  Winter Ale 

------------

Name:  Very Ill-Tempered Gnome (Barrel Aged) 
Style: Winter Ale 
ABV:  8% ABV 
IBU:  80 IBU 
Description:  various barrel aged varieties of our bigger version of our seasonal win 

------------

Name:  Coconut Edition B-Bomb (2017) 
Style: Winter Warmer 
ABV:  13% ABV 
IBU:  65 IBU 
Description:  This year’s release of B-Bomb is aged in 12-year-old American Oak bourbon barrels

Name:  Sigil IX 
Style: Winter Warmer 
ABV:  11.1% ABV 
IBU:  N/A IBU 
Description:  Sigil IX is a blended barrel-aged Winter Warmer Ale at 11.1% spiced with everything nice: white pepper, cardamom,  

------------

Name:  Eerie Von "Mother of Mercy" 
Style: Winter Warmer 
ABV:  9% ABV 
IBU:  50 IBU 
Description:  Apple Pie ALA Mode version of our eerie von spiced apple pie brown ale. At open hou 

------------

Name:  Kexmas Ale 
Style: Winter Warmer 
ABV:  7.6% ABV 
IBU:  N/A IBU 
Description:  Winter Warmer 

------------

Name:  Sandy Claus 
Style: Winter Warmer 
ABV:  7% ABV 
IBU:  18 IBU 
Description:  American red ale brewed with cinnamon, nutmeg,  

------------

Name:  Bourbon Barrel Aged White Death 
Style: Winter Warmer 
ABV:  6.2% ABV 
IBU:  N/A IBU 
Description:   

------------

Name:  Snow Angel 
Style: Winter Warmer 
ABV:  13% ABV 
IBU:  N/A IBU 
Description:  Winter Warmer 

------------

Name:  Zoigl-Pils 
Style: Zoigl 
ABV:  4.8% ABV 
IBU:  42 IBU 
Description:  Zoig

- Complete
- The below is cleaning the data before I save it to the csv

In [69]:
print(len(name_ls))
print(len(style_ls))
print(len(desc_ls))
print(len(abv_ls))
print(len(ibu_ls))

9205
9205
9205
9205
9205


In [70]:
#create df
df = pd.DataFrame()
df['Name'] = name_ls
df['style'] = style_ls
df['IBU'] = ibu_ls
df['ABV'] = abv_ls
df['Description'] = desc_ls
df

,Name,style,IBU,ABV,Description
0,Längst (Blend #1),Adambier,N/A IBU,10% ABV,Längst is our Bourbon barrel-aged Adambier - a...
1,Adam from the Wood,Adambier,N/A IBU,12% ABV,Adambier
2,Cherry Adam from the Wood,Adambier,N/A IBU,13.5% ABV,Cherry Adam from the Wood is aged with locally...
3,Adam from the Wood - Rye (2015),Adambier,N/A IBU,12.4% ABV,
4,Bishop's Barrel No. 17,Adambier,30 IBU,13.2% ABV,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
...,...,...,...,...,...
9200,Snow Angel,Winter Warmer,N/A IBU,13% ABV,Winter Warmer
9201,Zoigl-Pils,Zoigl,42 IBU,4.8% ABV,Zoigl
9202,ChiStar,Zoigl,23 IBU,5.5% ABV,Zoiglbier is a style that originated in a smal...
9203,Zoigl Bier,Zoigl,N/A IBU,4.8% ABV,


In [71]:
#get rid of all beers that don't have all 3 major things I need
df = df[(df['IBU']!='N/A IBU')&(df['ABV']!='N/A ABV')&(df['Description']!='')]

In [72]:
df

,Name,style,IBU,ABV,Description
4,Bishop's Barrel No. 17,Adambier,30 IBU,13.2% ABV,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
5,Adam,Adambier,50 IBU,10% ABV,Adambier
8,Dortmunder Adambier,Adambier,50 IBU,12.7% ABV,"A recreation of a historical German style, the..."
10,Arcanum (Singularis #1) (Hillrock Estate Soler...,Adambier,46 IBU,11% ABV,Adambier
11,Thurston Adambier,Adambier,18 IBU,12% ABV,Adambier
...,...,...,...,...,...
9196,"Eerie Von ""Mother of Mercy""",Winter Warmer,50 IBU,9% ABV,Apple Pie ALA Mode version of our eerie von sp...
9198,Sandy Claus,Winter Warmer,18 IBU,7% ABV,"American red ale brewed with cinnamon, nutmeg,"
9201,Zoigl-Pils,Zoigl,42 IBU,4.8% ABV,Zoigl
9202,ChiStar,Zoigl,23 IBU,5.5% ABV,Zoiglbier is a style that originated in a smal...


In [73]:
df['IBU'] = df['IBU'].str[:-4]
df['ABV'] = df['ABV'].str[:-5]
df['ABV'] = df['ABV'].astype(float)
df['IBU'] = df['IBU'].astype(int)

C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1187835521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].str[:-4]
C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1187835521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].str[:-5]
C:\Users\612456\AppData\Local\Temp\2\ipykernel_18876\1187835521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [74]:
df

,Name,style,IBU,ABV,Description
4,Bishop's Barrel No. 17,Adambier,30,13.2,Adambier Aged in Bourbon Barrels\n\nBishop’s B...
5,Adam,Adambier,50,10.0,Adambier
8,Dortmunder Adambier,Adambier,50,12.7,"A recreation of a historical German style, the..."
10,Arcanum (Singularis #1) (Hillrock Estate Soler...,Adambier,46,11.0,Adambier
11,Thurston Adambier,Adambier,18,12.0,Adambier
...,...,...,...,...,...
9196,"Eerie Von ""Mother of Mercy""",Winter Warmer,50,9.0,Apple Pie ALA Mode version of our eerie von sp...
9198,Sandy Claus,Winter Warmer,18,7.0,"American red ale brewed with cinnamon, nutmeg,"
9201,Zoigl-Pils,Zoigl,42,4.8,Zoigl
9202,ChiStar,Zoigl,23,5.5,Zoiglbier is a style that originated in a smal...


NEW CSV

In [75]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('continuous_beers.csv')

### Notes
I populated the new dataframe and then combined it with the old one, deleted repeated values and saved it to untappd_beers.csv